# Polynomial Regression Models

## CONTENTS

__1.LIBRARIES AND DATA__
    
_Importing Necessary tools for data science and dataset_
        
__2.DEFINING FUNCTIONS__

_Writing functions that will be frequently used_

__3.ADDING POLYNOMIAL FEATURES__

_Expanding the number of predictors by adding Polynomial Features of_

__4.APPLYING FEATURE SELECTION METHODS TO POLYNOMIAL DATAFRAMES__

    4.1 Applying Features Selection Methods to X = X^1 + X^2 dataset
    4.2 Applying Features Selection Methods to X = X^1 + X^2 + X^3 dataset

__5.APPLYING THE SAME STRATEGY FROM PART 4 TO A REDUCED DATASET__

    5.1 Reduce the dataset via forward selection
    5.2 Add Polynomial features to the reduced dataset
    5.3 Run MLR models and store results
    5.4 Further reduce the dataset via all the feature selecting methods
    5.5 Run MLR models and store results
    
__6.CONCLUSION__
    

## 1. LIBRARIES AND DATA

__Libraries__

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

pd.set_option('display.max_columns', 100)

__Importing Data__

In [2]:
cancer_df = pd.read_csv(r'C:\Users\Constantine\OneDrive\Υπολογιστής\projects\Predicting Death Rate From Cancer\cancer_reg_refined.csv',
                 encoding='latin-1') 
cancer_df.head(2)

# Without the encoding paremeter, this error presents itself: UnicodeDecodeError: 'utf-8' codec can't decode 
# byte 0xf1 in position 41137: invalid continuation byte

,avgAnnCount,avgDeathsPerYear,TARGET_deathRate,incidenceRate,medIncome,popEst2015,povertyPercent,studyPerCap,binnedInc,MedianAge,MedianAgeMale,MedianAgeFemale,Geography,AvgHouseholdSize,PercentMarried,PctNoHS18_24,PctHS18_24,PctBachDeg18_24,PctHS25_Over,PctBachDeg25_Over,PctEmployed16_Over,PctUnemployed16_Over,PctPrivateCoverage,PctEmpPrivCoverage,PctPublicCoverage,PctPublicCoverageAlone,PctWhite,PctBlack,PctAsian,PctOtherRace,PctMarriedHouseholds,BirthRate
0,1397.0,469,164.9,489.8,61898,260131,11.2,499.748204,93564.75,39.3,36.9,41.7,WEST,2.54,52.5,11.5,39.5,6.9,23.2,19.6,51.9,8.0,75.1,41.6,32.9,14.0,81.780529,2.594728,4.821857,1.843479,52.856076,6.118831
1,173.0,70,161.3,411.6,48127,43269,18.6,23.111234,49534.00,33.0,32.2,33.7,WEST,2.34,44.5,6.1,22.4,7.5,26.0,22.7,55.9,7.8,70.2,43.6,31.1,15.3,89.228509,0.969102,2.246233,3.741352,45.372500,4.333096


In [3]:
# Copying and shuffling the original Dataframe
df1 = cancer_df.sample(frac = 1) 

In [4]:
# Getting Dummy variables
df1 = pd.get_dummies(data = df1, drop_first = True)
df1.head(2) 

,avgAnnCount,avgDeathsPerYear,TARGET_deathRate,incidenceRate,medIncome,popEst2015,povertyPercent,studyPerCap,binnedInc,MedianAge,MedianAgeMale,MedianAgeFemale,AvgHouseholdSize,PercentMarried,PctNoHS18_24,PctHS18_24,PctBachDeg18_24,PctHS25_Over,PctBachDeg25_Over,PctEmployed16_Over,PctUnemployed16_Over,PctPrivateCoverage,PctEmpPrivCoverage,PctPublicCoverage,PctPublicCoverageAlone,PctWhite,PctBlack,PctAsian,PctOtherRace,PctMarriedHouseholds,BirthRate,Geography_EAST,Geography_WEST
1146,1962.667684,16,151.3,453.549422,48645,6282,11.2,0.000000,49534.00,44.4,40.8,48.0,2.25,59.8,16.5,40.7,4.2,34.0,18.0,60.5,2.6,79.5,47.3,27.6,11.3,97.871665,0.381194,0.698856,0.142948,55.303312,2.550091,0,0
144,22.000000,11,166.3,357.000000,34304,4423,24.5,678.272666,35815.95,41.8,40.6,42.4,2.47,46.1,23.7,48.5,0.7,31.0,9.3,50.0,12.2,42.7,23.4,51.9,35.0,91.212578,0.258454,0.000000,5.104458,44.584245,5.485232,0,1


## 2. DEFINING FUNCTIONS 

In [5]:
# Defining forward selection function
def forward_selection(data, target, significance_level):
    initial_features = data.columns.tolist()
    best_features = []
    while (len(initial_features)>0):
        remaining_features = list(set(initial_features)-set(best_features))
        new_pval = pd.Series(index=remaining_features)
        for new_column in remaining_features:
            model = sm.OLS(target, sm.add_constant(data[best_features+[new_column]])).fit()
            new_pval[new_column] = model.pvalues[new_column]
        min_p_value = new_pval.min()
        if(min_p_value<significance_level):
            best_features.append(new_pval.idxmin())
        else:
            break
    return best_features


# Defining backward elimination function
def backward_elimination(data, target,significance_level):
    features = data.columns.tolist()
    while(len(features)>0):
        features_with_constant = sm.add_constant(data[features])
        p_values = sm.OLS(target, features_with_constant).fit().pvalues[1:]
        max_p_value = p_values.max()
        if(max_p_value >= significance_level):
            excluded_feature = p_values.idxmax()
            features.remove(excluded_feature)
        else:
            break 
    return features


# Defining the stepwise selection function
def stepwise_selection(data, target,SL_in,SL_out):
    initial_features = data.columns.tolist()
    best_features = []
    while (len(initial_features)>0):
        remaining_features = list(set(initial_features)-set(best_features))
        new_pval = pd.Series(index=remaining_features)
        for new_column in remaining_features:
            model = sm.OLS(target, sm.add_constant(data[best_features+[new_column]])).fit()
            new_pval[new_column] = model.pvalues[new_column]
        min_p_value = new_pval.min()
        if(min_p_value<SL_in):
            best_features.append(new_pval.idxmin())
            while(len(best_features)>0):
                best_features_with_constant = sm.add_constant(data[best_features])
                p_values = sm.OLS(target, best_features_with_constant).fit().pvalues[1:]
                max_p_value = p_values.max()
                if(max_p_value >= SL_out):
                    excluded_feature = p_values.idxmax()
                    best_features.remove(excluded_feature)
                else:
                    break 
        else:
            break
    return best_features


# Using the RepeatedKFold function for cross-validation
rkf = RepeatedKFold(n_splits= 5, n_repeats= 10, random_state= 126) 


# Creating function that calculates cross-validated scores
def cv_scores(X,Y):
        
    cv_MAE = round(np.mean(cross_val_score(LinearRegression(), X, Y, cv=rkf, 
                                       scoring='neg_mean_absolute_error', error_score='raise') * (-1)),3)

    cv_MSE = round(np.mean(cross_val_score(LinearRegression(), X, Y, cv=rkf, 
                                          scoring='neg_mean_squared_error', error_score='raise') * (-1)),1)
    
    cv_MAPE = round(np.mean(cross_val_score(LinearRegression(), X, Y, cv=rkf, 
                                          scoring='neg_mean_absolute_percentage_error', error_score='raise') * (-100)),3)
    
    cv_R2 = round(np.mean(cross_val_score(LinearRegression(), X, Y, cv=rkf, 
                                          scoring='r2', error_score='raise')),3)
    
    cv_F_stat = round(((cv_R2)/(1-cv_R2))*((len(X) - len(X.columns) - 1 )/len(X.columns)),2)
    
    score_list = [cv_MAE, cv_MSE,cv_MAPE,cv_R2,cv_F_stat,len(X.columns)]
    return score_list

## 3. ADDING POLYNOMIAL FEATURES 

In [6]:
# Defining X and Y

Y = df1['TARGET_deathRate']
X = df1.drop('TARGET_deathRate', axis = 1)
X.head(2)

,avgAnnCount,avgDeathsPerYear,incidenceRate,medIncome,popEst2015,povertyPercent,studyPerCap,binnedInc,MedianAge,MedianAgeMale,MedianAgeFemale,AvgHouseholdSize,PercentMarried,PctNoHS18_24,PctHS18_24,PctBachDeg18_24,PctHS25_Over,PctBachDeg25_Over,PctEmployed16_Over,PctUnemployed16_Over,PctPrivateCoverage,PctEmpPrivCoverage,PctPublicCoverage,PctPublicCoverageAlone,PctWhite,PctBlack,PctAsian,PctOtherRace,PctMarriedHouseholds,BirthRate,Geography_EAST,Geography_WEST
1146,1962.667684,16,453.549422,48645,6282,11.2,0.000000,49534.00,44.4,40.8,48.0,2.25,59.8,16.5,40.7,4.2,34.0,18.0,60.5,2.6,79.5,47.3,27.6,11.3,97.871665,0.381194,0.698856,0.142948,55.303312,2.550091,0,0
144,22.000000,11,357.000000,34304,4423,24.5,678.272666,35815.95,41.8,40.6,42.4,2.47,46.1,23.7,48.5,0.7,31.0,9.3,50.0,12.2,42.7,23.4,51.9,35.0,91.212578,0.258454,0.000000,5.104458,44.584245,5.485232,0,1


In [7]:
# Dividing X to numeric and categorical dataframes

X_num = X.drop(columns = ['Geography_EAST','Geography_WEST'])
X_cat = X[['Geography_EAST','Geography_WEST']]

# There is no point to addying categorical features to power of n to the dataset

__Creating the X^2 and X^3 dataframes__

In [8]:
# Copying the numeric dataframe and assigning it to a variable.
X2_num = X_num.copy()  

In [9]:
# Loop that adds polynomial feature of 2nd degree.
for i in X_num.columns:
    
    name2 = i + '^2'
    
    Z2 = X_num[i] * X_num[i]
    
    X2_num[name2] = Z2

In [10]:
# Copying the numeric dataframe and assigning it to a variable.
X3_num = X2_num.copy()

In [11]:
# Loop that adds polynomial feature of 3rd degree.
for i in X_num.columns:
    
    name3 = i + '^3'
    
    Z3 = X_num[i] * X_num[i] * X_num[i]
   
    X3_num[name3] = Z3

In [12]:
# Concatenating numeric and categorical dataframes
X2 = pd.concat([X2_num.round(3), X_cat], axis =  1)
X3 = pd.concat([X3_num.round(3), X_cat], axis =  1)

In [13]:
# Independent dataframe is X + X^2
X2.head(2)

,avgAnnCount,avgDeathsPerYear,incidenceRate,medIncome,popEst2015,povertyPercent,studyPerCap,binnedInc,MedianAge,MedianAgeMale,MedianAgeFemale,AvgHouseholdSize,PercentMarried,PctNoHS18_24,PctHS18_24,PctBachDeg18_24,PctHS25_Over,PctBachDeg25_Over,PctEmployed16_Over,PctUnemployed16_Over,PctPrivateCoverage,PctEmpPrivCoverage,PctPublicCoverage,PctPublicCoverageAlone,PctWhite,PctBlack,PctAsian,PctOtherRace,PctMarriedHouseholds,BirthRate,avgAnnCount^2,avgDeathsPerYear^2,incidenceRate^2,medIncome^2,popEst2015^2,povertyPercent^2,studyPerCap^2,binnedInc^2,MedianAge^2,MedianAgeMale^2,MedianAgeFemale^2,AvgHouseholdSize^2,PercentMarried^2,PctNoHS18_24^2,PctHS18_24^2,PctBachDeg18_24^2,PctHS25_Over^2,PctBachDeg25_Over^2,PctEmployed16_Over^2,PctUnemployed16_Over^2,PctPrivateCoverage^2,PctEmpPrivCoverage^2,PctPublicCoverage^2,PctPublicCoverageAlone^2,PctWhite^2,PctBlack^2,PctAsian^2,PctOtherRace^2,PctMarriedHouseholds^2,BirthRate^2,Geography_EAST,Geography_WEST
1146,1962.668,16,453.549,48645,6282,11.2,0.000,49534.00,44.4,40.8,48.0,2.25,59.8,16.5,40.7,4.2,34.0,18.0,60.5,2.6,79.5,47.3,27.6,11.3,97.872,0.381,0.699,0.143,55.303,2.550,3852064.438,256,205707.078,2366336025,39463524,125.44,0.000,2.453617e+09,1971.36,1664.64,2304.00,5.062,3576.04,272.25,1656.49,17.64,1156.0,324.00,3660.25,6.76,6320.25,2237.29,761.76,127.69,9578.863,0.145,0.488,0.020,3058.456,6.503,0,0
144,22.000,11,357.000,34304,4423,24.5,678.273,35815.95,41.8,40.6,42.4,2.47,46.1,23.7,48.5,0.7,31.0,9.3,50.0,12.2,42.7,23.4,51.9,35.0,91.213,0.258,0.000,5.104,44.584,5.485,484.000,121,127449.000,1176764416,19562929,600.25,460053.809,1.282782e+09,1747.24,1648.36,1797.76,6.101,2125.21,561.69,2352.25,0.49,961.0,86.49,2500.00,148.84,1823.29,547.56,2693.61,1225.00,8319.734,0.067,0.000,26.055,1987.755,30.088,0,1


In [14]:
# Independent dataframe is X + X^2 + X^3
X3.head(2)

,avgAnnCount,avgDeathsPerYear,incidenceRate,medIncome,popEst2015,povertyPercent,studyPerCap,binnedInc,MedianAge,MedianAgeMale,MedianAgeFemale,AvgHouseholdSize,PercentMarried,PctNoHS18_24,PctHS18_24,PctBachDeg18_24,PctHS25_Over,PctBachDeg25_Over,PctEmployed16_Over,PctUnemployed16_Over,PctPrivateCoverage,PctEmpPrivCoverage,PctPublicCoverage,PctPublicCoverageAlone,PctWhite,PctBlack,PctAsian,PctOtherRace,PctMarriedHouseholds,BirthRate,avgAnnCount^2,avgDeathsPerYear^2,incidenceRate^2,medIncome^2,popEst2015^2,povertyPercent^2,studyPerCap^2,binnedInc^2,MedianAge^2,MedianAgeMale^2,MedianAgeFemale^2,AvgHouseholdSize^2,PercentMarried^2,PctNoHS18_24^2,PctHS18_24^2,PctBachDeg18_24^2,PctHS25_Over^2,PctBachDeg25_Over^2,PctEmployed16_Over^2,PctUnemployed16_Over^2,PctPrivateCoverage^2,PctEmpPrivCoverage^2,PctPublicCoverage^2,PctPublicCoverageAlone^2,PctWhite^2,PctBlack^2,PctAsian^2,PctOtherRace^2,PctMarriedHouseholds^2,BirthRate^2,avgAnnCount^3,avgDeathsPerYear^3,incidenceRate^3,medIncome^3,popEst2015^3,povertyPercent^3,studyPerCap^3,binnedInc^3,MedianAge^3,MedianAgeMale^3,MedianAgeFemale^3,AvgHouseholdSize^3,PercentMarried^3,PctNoHS18_24^3,PctHS18_24^3,PctBachDeg18_24^3,PctHS25_Over^3,PctBachDeg25_Over^3,PctEmployed16_Over^3,PctUnemployed16_Over^3,PctPrivateCoverage^3,PctEmpPrivCoverage^3,PctPublicCoverage^3,PctPublicCoverageAlone^3,PctWhite^3,PctBlack^3,PctAsian^3,PctOtherRace^3,PctMarriedHouseholds^3,BirthRate^3,Geography_EAST,Geography_WEST
1146,1962.668,16,453.549,48645,6282,11.2,0.000,49534.00,44.4,40.8,48.0,2.25,59.8,16.5,40.7,4.2,34.0,18.0,60.5,2.6,79.5,47.3,27.6,11.3,97.872,0.381,0.699,0.143,55.303,2.550,3852064.438,256,205707.078,2366336025,39463524,125.44,0.000,2.453617e+09,1971.36,1664.64,2304.00,5.062,3576.04,272.25,1656.49,17.64,1156.0,324.00,3660.25,6.76,6320.25,2237.29,761.76,127.69,9578.863,0.145,0.488,0.020,3058.456,6.503,7.560322e+09,4096,9.329833e+07,115110415936125,247909857768,1404.928,0.000000e+00,1.215375e+14,87528.384,67917.312,110592.000,11.391,213847.192,4492.125,67419.143,74.088,39304.0,5832.000,221445.125,17.576,502459.875,105823.817,21024.576,1442.897,937499.239,0.055,0.341,0.003,169142.766,16.583,0,0
144,22.000,11,357.000,34304,4423,24.5,678.273,35815.95,41.8,40.6,42.4,2.47,46.1,23.7,48.5,0.7,31.0,9.3,50.0,12.2,42.7,23.4,51.9,35.0,91.213,0.258,0.000,5.104,44.584,5.485,484.000,121,127449.000,1176764416,19562929,600.25,460053.809,1.282782e+09,1747.24,1648.36,1797.76,6.101,2125.21,561.69,2352.25,0.49,961.0,86.49,2500.00,148.84,1823.29,547.56,2693.61,1225.00,8319.734,0.067,0.000,26.055,1987.755,30.088,1.064800e+04,1331,4.549929e+07,40367726526464,86526834967,14706.125,3.120419e+08,4.594407e+13,73034.632,66923.416,76225.024,15.069,97972.181,13312.053,114084.125,0.343,29791.0,804.357,125000.000,1815.848,77854.483,12812.904,139798.359,42875.000,758864.423,0.017,0.000,132.999,88622.552,165.038,0,1


In [15]:
# Storing the cross-validated scores

CV_RESULTS = pd.DataFrame({'Pol.F_2deg':cv_scores(X2,Y)}, index=['MAE','MSE', 'MAPE','R^2','F-stat','N_Preds'])


CV_RESULTS = pd.concat([CV_RESULTS ,pd.DataFrame({'Pol.F_3deg':cv_scores(X3,Y)},
                                                 index=['MAE','MSE', 'MAPE','R^2','F-stat','N_Preds'])], axis =  1)

CV_RESULTS

,Pol.F_2deg,Pol.F_3deg
MAE,13.359,25.423
MSE,332.600,1981.100
MAPE,7.744,14.914
R^2,0.563,-1.663
F-stat,54.380,-17.560
N_Preds,62.000,92.000


## 4. APPLYING FEATURE SELECTION METHODS TO POLYNOMIAL DATAFRAMES

__4.1 Applying Features Selection Methods to X = X^1 + X^2 dataset__

In [16]:
# Reducing X with Forward Selection
X2_f = X2[forward_selection(X2,Y,0.01)]

In [17]:
X2_f.head(2)

,PctBachDeg25_Over,incidenceRate,povertyPercent,Geography_WEST,PctHS18_24,PctPrivateCoverage^2,PctOtherRace,PctEmpPrivCoverage^2,PctEmployed16_Over^2,MedianAgeFemale^2,PctMarriedHouseholds,PercentMarried,BirthRate,PctOtherRace^2,avgAnnCount,medIncome^2,PctPrivateCoverage,PctPublicCoverageAlone^2,PctPublicCoverageAlone,PctBlack^2,PctWhite,medIncome
1146,18.0,453.549,11.2,0,40.7,6320.25,0.143,2237.29,3660.25,2304.00,55.303,59.8,2.550,0.020,1962.668,2366336025,79.5,127.69,11.3,0.145,97.872,48645
144,9.3,357.000,24.5,1,48.5,1823.29,5.104,547.56,2500.00,1797.76,44.584,46.1,5.485,26.055,22.000,1176764416,42.7,1225.00,35.0,0.067,91.213,34304


In [18]:
# Reducing X with Backward Elimination
X2_b = X2[backward_elimination(X2,Y,0.01)]

In [19]:
X2_b.head(2)

,avgAnnCount,avgDeathsPerYear,incidenceRate,popEst2015,binnedInc,PercentMarried,PctHS18_24,PctBachDeg25_Over,PctEmployed16_Over,PctPrivateCoverage,PctEmpPrivCoverage,PctPublicCoverage,PctPublicCoverageAlone,PctWhite,PctOtherRace,PctMarriedHouseholds,BirthRate,avgAnnCount^2,avgDeathsPerYear^2,medIncome^2,popEst2015^2,binnedInc^2,MedianAge^2,PercentMarried^2,PctPrivateCoverage^2,PctEmpPrivCoverage^2,PctPublicCoverageAlone^2,PctWhite^2,PctBlack^2,PctOtherRace^2,PctMarriedHouseholds^2,Geography_WEST
1146,1962.668,16,453.549,6282,49534.00,59.8,40.7,18.0,60.5,79.5,47.3,27.6,11.3,97.872,0.143,55.303,2.550,3852064.438,256,2366336025,39463524,2.453617e+09,1971.36,3576.04,6320.25,2237.29,127.69,9578.863,0.145,0.020,3058.456,0
144,22.000,11,357.000,4423,35815.95,46.1,48.5,9.3,50.0,42.7,23.4,51.9,35.0,91.213,5.104,44.584,5.485,484.000,121,1176764416,19562929,1.282782e+09,1747.24,2125.21,1823.29,547.56,1225.00,8319.734,0.067,26.055,1987.755,1


In [20]:
# Reducing X with Stepwise Selection
X2_s = X2[stepwise_selection(X2,Y,0.01,0.01)]

In [21]:
X2_s.head(2)

,PctBachDeg25_Over,incidenceRate,Geography_WEST,PctHS18_24,PctPrivateCoverage^2,PctOtherRace,PctEmpPrivCoverage^2,PctEmployed16_Over^2,MedianAgeFemale^2,PctMarriedHouseholds,PercentMarried,BirthRate,PctOtherRace^2,avgAnnCount
1146,18.0,453.549,0,40.7,6320.25,0.143,2237.29,3660.25,2304.00,55.303,59.8,2.550,0.020,1962.668
144,9.3,357.000,1,48.5,1823.29,5.104,547.56,2500.00,1797.76,44.584,46.1,5.485,26.055,22.000


In [22]:
# Storing the cross-validated scores

CV_RESULTS = pd.concat([CV_RESULTS ,pd.DataFrame({'2deg_reduced_by_f':cv_scores(X2_f,Y)},
                                                 index=['MAE','MSE', 'MAPE','R^2','F-stat','N_Preds'])], axis =  1)

CV_RESULTS = pd.concat([CV_RESULTS ,pd.DataFrame({'2deg_reduced_by_b':cv_scores(X2_b,Y)},
                                                 index=['MAE','MSE', 'MAPE','R^2','F-stat','N_Preds'])], axis =  1)

CV_RESULTS = pd.concat([CV_RESULTS ,pd.DataFrame({'2deg_reduced_by_s':cv_scores(X2_s,Y)},
                                                 index=['MAE','MSE', 'MAPE','R^2','F-stat','N_Preds'])], axis =  1)

CV_RESULTS

,Pol.F_2deg,Pol.F_3deg,2deg_reduced_by_f,2deg_reduced_by_b,2deg_reduced_by_s
MAE,13.359,25.423,14.015,13.199,14.159
MSE,332.600,1981.100,358.800,323.200,367.400
MAPE,7.744,14.914,8.098,7.645,8.177
R^2,0.563,-1.663,0.529,0.575,0.518
F-stat,54.380,-17.560,135.640,111.910,204.570
N_Preds,62.000,92.000,22.000,32.000,14.000


__4.2 Applying Features Selection Methods to X = X^1 + X^2 + X^3 dataset__

In [23]:
# Reducing X with Forward Selection
X3_f = X3[forward_selection(X3,Y, 0.005)]

In [24]:
# Reducing X with Backward Elimination
X3_b = X3[backward_elimination(X3,Y,0.005)]

We will not be using the stepwise function because the calculation time is quite large!

In [25]:
# Storing the cross-validated scores

CV_RESULTS = pd.concat([CV_RESULTS ,pd.DataFrame({'3deg_reduced_by_f':cv_scores(X3_f,Y)},
                                                 index=['MAE','MSE', 'MAPE','R^2','F-stat','N_Preds'])], axis =  1)

CV_RESULTS = pd.concat([CV_RESULTS ,pd.DataFrame({'3deg_reduced_by_b':cv_scores(X3_b,Y)},
                                                 index=['MAE','MSE', 'MAPE','R^2','F-stat','N_Preds'])], axis =  1)


CV_RESULTS

,Pol.F_2deg,Pol.F_3deg,2deg_reduced_by_f,2deg_reduced_by_b,2deg_reduced_by_s,3deg_reduced_by_f,3deg_reduced_by_b
MAE,13.359,25.423,14.015,13.199,14.159,17.626,20.192
MSE,332.600,1981.100,358.800,323.200,367.400,599.800,1273.700
MAPE,7.744,14.914,8.098,7.645,8.177,10.183,11.625
R^2,0.563,-1.663,0.529,0.575,0.518,0.214,-0.670
F-stat,54.380,-17.560,135.640,111.910,204.570,13.490,-25.190
N_Preds,62.000,92.000,22.000,32.000,14.000,53.000,42.000


## 5. APPLYING THE SAME STRATEGY FROM PART 4 TO A REDUCED DATASET

We remember that if we applied any feature selection methods to the original dataset, we would end with the same predictors.
The strategy that we are going to apply is:

-  Step 1: Reduce the dataset via forward selection
-  Step 2: Add Polynomial features to the reduced dataset
-  Step 3: Run MLR models and store results
-  Step 4: Further reduce the dataset via all the feature selecting methods
-  Step 5: Run MLR models and store results

__5.1. Reduce the dataset via forward selection__

In [26]:
# Reducing the dataset via forward selection
X_f = X[forward_selection(X,Y, 0.01)]

In [27]:
X_f.head(2)

,PctBachDeg25_Over,incidenceRate,povertyPercent,Geography_WEST,PctHS18_24,PctUnemployed16_Over,PctPrivateCoverage,PctOtherRace,PctEmpPrivCoverage,BirthRate,PctMarriedHouseholds,avgAnnCount,PctBlack,MedianAgeFemale,PctEmployed16_Over,PercentMarried,PctWhite
1146,18.0,453.549422,11.2,0,40.7,2.6,79.5,0.142948,47.3,2.550091,55.303312,1962.667684,0.381194,48.0,60.5,59.8,97.871665
144,9.3,357.000000,24.5,1,48.5,12.2,42.7,5.104458,23.4,5.485232,44.584245,22.000000,0.258454,42.4,50.0,46.1,91.212578


__5.2. Addying Polynomial Features to the reduced dataset__

In [28]:
# Copying the dataframe
X_f_2 = X_f.copy()

In [29]:
for i in X_f.columns:
    
    name2 = i + '^2'
    
    Z2 = X_f[i] * X_f[i]
    
    X_f_2[name2] = Z2

In [30]:
# Independent dataframe is X + X^2
X_f_2.head(2)

,PctBachDeg25_Over,incidenceRate,povertyPercent,Geography_WEST,PctHS18_24,PctUnemployed16_Over,PctPrivateCoverage,PctOtherRace,PctEmpPrivCoverage,BirthRate,PctMarriedHouseholds,avgAnnCount,PctBlack,MedianAgeFemale,PctEmployed16_Over,PercentMarried,PctWhite,PctBachDeg25_Over^2,incidenceRate^2,povertyPercent^2,Geography_WEST^2,PctHS18_24^2,PctUnemployed16_Over^2,PctPrivateCoverage^2,PctOtherRace^2,PctEmpPrivCoverage^2,BirthRate^2,PctMarriedHouseholds^2,avgAnnCount^2,PctBlack^2,MedianAgeFemale^2,PctEmployed16_Over^2,PercentMarried^2,PctWhite^2
1146,18.0,453.549422,11.2,0,40.7,2.6,79.5,0.142948,47.3,2.550091,55.303312,1962.667684,0.381194,48.0,60.5,59.8,97.871665,324.00,205707.078287,125.44,0,1656.49,6.76,6320.25,0.020434,2237.29,6.502964,3058.456345,3.852064e+06,0.145309,2304.00,3660.25,3576.04,9578.862722
144,9.3,357.000000,24.5,1,48.5,12.2,42.7,5.104458,23.4,5.485232,44.584245,22.000000,0.258454,42.4,50.0,46.1,91.212578,86.49,127449.000000,600.25,1,2352.25,148.84,1823.29,26.055495,547.56,30.087771,1987.754909,4.840000e+02,0.066798,1797.76,2500.00,2125.21,8319.734398


In [31]:
# Deleting the redundant variables
X_f_2 = X_f_2.drop('Geography_WEST^2', axis = 1)

In [32]:
# Checking again the Independent dataframe
X_f_2.head(2)

,PctBachDeg25_Over,incidenceRate,povertyPercent,Geography_WEST,PctHS18_24,PctUnemployed16_Over,PctPrivateCoverage,PctOtherRace,PctEmpPrivCoverage,BirthRate,PctMarriedHouseholds,avgAnnCount,PctBlack,MedianAgeFemale,PctEmployed16_Over,PercentMarried,PctWhite,PctBachDeg25_Over^2,incidenceRate^2,povertyPercent^2,PctHS18_24^2,PctUnemployed16_Over^2,PctPrivateCoverage^2,PctOtherRace^2,PctEmpPrivCoverage^2,BirthRate^2,PctMarriedHouseholds^2,avgAnnCount^2,PctBlack^2,MedianAgeFemale^2,PctEmployed16_Over^2,PercentMarried^2,PctWhite^2
1146,18.0,453.549422,11.2,0,40.7,2.6,79.5,0.142948,47.3,2.550091,55.303312,1962.667684,0.381194,48.0,60.5,59.8,97.871665,324.00,205707.078287,125.44,1656.49,6.76,6320.25,0.020434,2237.29,6.502964,3058.456345,3.852064e+06,0.145309,2304.00,3660.25,3576.04,9578.862722
144,9.3,357.000000,24.5,1,48.5,12.2,42.7,5.104458,23.4,5.485232,44.584245,22.000000,0.258454,42.4,50.0,46.1,91.212578,86.49,127449.000000,600.25,2352.25,148.84,1823.29,26.055495,547.56,30.087771,1987.754909,4.840000e+02,0.066798,1797.76,2500.00,2125.21,8319.734398


In [33]:
# Copying the dataframe
X_f_3 = X_f_2.copy()

In [34]:
for i in X_f.columns:
    
    name3 = i + '^3'
    
    Z3 = X_f[i] * X_f[i] * X_f[i]
   
    X_f_3[name3] = Z3

In [35]:
# Independent dataframe is X + X^2 + X^3
X_f_3.head(2)

,PctBachDeg25_Over,incidenceRate,povertyPercent,Geography_WEST,PctHS18_24,PctUnemployed16_Over,PctPrivateCoverage,PctOtherRace,PctEmpPrivCoverage,BirthRate,PctMarriedHouseholds,avgAnnCount,PctBlack,MedianAgeFemale,PctEmployed16_Over,PercentMarried,PctWhite,PctBachDeg25_Over^2,incidenceRate^2,povertyPercent^2,PctHS18_24^2,PctUnemployed16_Over^2,PctPrivateCoverage^2,PctOtherRace^2,PctEmpPrivCoverage^2,BirthRate^2,PctMarriedHouseholds^2,avgAnnCount^2,PctBlack^2,MedianAgeFemale^2,PctEmployed16_Over^2,PercentMarried^2,PctWhite^2,PctBachDeg25_Over^3,incidenceRate^3,povertyPercent^3,Geography_WEST^3,PctHS18_24^3,PctUnemployed16_Over^3,PctPrivateCoverage^3,PctOtherRace^3,PctEmpPrivCoverage^3,BirthRate^3,PctMarriedHouseholds^3,avgAnnCount^3,PctBlack^3,MedianAgeFemale^3,PctEmployed16_Over^3,PercentMarried^3,PctWhite^3
1146,18.0,453.549422,11.2,0,40.7,2.6,79.5,0.142948,47.3,2.550091,55.303312,1962.667684,0.381194,48.0,60.5,59.8,97.871665,324.00,205707.078287,125.44,1656.49,6.76,6320.25,0.020434,2237.29,6.502964,3058.456345,3.852064e+06,0.145309,2304.00,3660.25,3576.04,9578.862722,5832.000,9.329833e+07,1404.928,0,67419.143,17.576,502459.875,0.002921,105823.817,16.583152,169142.766204,7.560322e+09,0.055391,110592.000,221445.125,213847.192,937499.239077
144,9.3,357.000000,24.5,1,48.5,12.2,42.7,5.104458,23.4,5.485232,44.584245,22.000000,0.258454,42.4,50.0,46.1,91.212578,86.49,127449.000000,600.25,2352.25,148.84,1823.29,26.055495,547.56,30.087771,1987.754909,4.840000e+02,0.066798,1797.76,2500.00,2125.21,8319.734398,804.357,4.549929e+07,14706.125,1,114084.125,1815.848,77854.483,132.999187,12812.904,165.038405,88622.552038,1.064800e+04,0.017264,76225.024,125000.000,97972.181,758864.423315


In [36]:
# Deleting the redundant variables
X_f_3 = X_f_3.drop('Geography_WEST^3', axis = 1)

In [37]:
# Checking again the Independent dataframe
X_f_3.head(2)

,PctBachDeg25_Over,incidenceRate,povertyPercent,Geography_WEST,PctHS18_24,PctUnemployed16_Over,PctPrivateCoverage,PctOtherRace,PctEmpPrivCoverage,BirthRate,PctMarriedHouseholds,avgAnnCount,PctBlack,MedianAgeFemale,PctEmployed16_Over,PercentMarried,PctWhite,PctBachDeg25_Over^2,incidenceRate^2,povertyPercent^2,PctHS18_24^2,PctUnemployed16_Over^2,PctPrivateCoverage^2,PctOtherRace^2,PctEmpPrivCoverage^2,BirthRate^2,PctMarriedHouseholds^2,avgAnnCount^2,PctBlack^2,MedianAgeFemale^2,PctEmployed16_Over^2,PercentMarried^2,PctWhite^2,PctBachDeg25_Over^3,incidenceRate^3,povertyPercent^3,PctHS18_24^3,PctUnemployed16_Over^3,PctPrivateCoverage^3,PctOtherRace^3,PctEmpPrivCoverage^3,BirthRate^3,PctMarriedHouseholds^3,avgAnnCount^3,PctBlack^3,MedianAgeFemale^3,PctEmployed16_Over^3,PercentMarried^3,PctWhite^3
1146,18.0,453.549422,11.2,0,40.7,2.6,79.5,0.142948,47.3,2.550091,55.303312,1962.667684,0.381194,48.0,60.5,59.8,97.871665,324.00,205707.078287,125.44,1656.49,6.76,6320.25,0.020434,2237.29,6.502964,3058.456345,3.852064e+06,0.145309,2304.00,3660.25,3576.04,9578.862722,5832.000,9.329833e+07,1404.928,67419.143,17.576,502459.875,0.002921,105823.817,16.583152,169142.766204,7.560322e+09,0.055391,110592.000,221445.125,213847.192,937499.239077
144,9.3,357.000000,24.5,1,48.5,12.2,42.7,5.104458,23.4,5.485232,44.584245,22.000000,0.258454,42.4,50.0,46.1,91.212578,86.49,127449.000000,600.25,2352.25,148.84,1823.29,26.055495,547.56,30.087771,1987.754909,4.840000e+02,0.066798,1797.76,2500.00,2125.21,8319.734398,804.357,4.549929e+07,14706.125,114084.125,1815.848,77854.483,132.999187,12812.904,165.038405,88622.552038,1.064800e+04,0.017264,76225.024,125000.000,97972.181,758864.423315


__5.3. Running MLR models and storing results__

In [38]:
# Storing the cross-validated scores

CV_RESULTS = pd.concat([CV_RESULTS ,pd.DataFrame({'df_reduced_2deg':cv_scores(X_f_2,Y)},
                                                 index=['MAE','MSE', 'MAPE','R^2','F-stat','N_Preds'])], axis =  1)

CV_RESULTS = pd.concat([CV_RESULTS ,pd.DataFrame({'df_reduced_3deg':cv_scores(X_f_3,Y)},
                                                 index=['MAE','MSE', 'MAPE','R^2','F-stat','N_Preds'])], axis =  1)


CV_RESULTS

,Pol.F_2deg,Pol.F_3deg,2deg_reduced_by_f,2deg_reduced_by_b,2deg_reduced_by_s,3deg_reduced_by_f,3deg_reduced_by_b,df_reduced_2deg,df_reduced_3deg
MAE,13.359,25.423,14.015,13.199,14.159,17.626,20.192,14.113,14.096
MSE,332.600,1981.100,358.800,323.200,367.400,599.800,1273.700,366.100,368.400
MAPE,7.744,14.914,8.098,7.645,8.177,10.183,11.625,8.148,8.128
R^2,0.563,-1.663,0.529,0.575,0.518,0.214,-0.670,0.520,0.516
F-stat,54.380,-17.560,135.640,111.910,204.570,13.490,-25.190,86.860,57.220
N_Preds,62.000,92.000,22.000,32.000,14.000,53.000,42.000,33.000,49.000


__5.4. Further reduce the dataset via all the feature selecting methods__

In [39]:
# Reducing the Reduced Dataset with Polynomial Features of 2nd degree via forward selection
X_f_2_f = X_f_2[forward_selection(X_f_2,Y, 0.01)]

In [40]:
# Reducing the Reduced Dataset with Polynomial Features of 2nd degree via backward elimination
X_f_2_b = X_f_2[backward_elimination(X_f_2,Y,0.01)]

In [41]:
# Reducing the Reduced Dataset with Polynomial Features of 2nd degree via stepwise selection
X_f_2_s = X_f_2[stepwise_selection(X_f_2,Y,0.01, 0.01)]

In [42]:
# Storing the cross-validated scores

CV_RESULTS = pd.concat([CV_RESULTS ,pd.DataFrame({'df_reduced_2deg_reduced_f':cv_scores(X_f_2_f,Y)},
                                                 index=['MAE','MSE', 'MAPE','R^2','F-stat','N_Preds'])], axis =  1)

CV_RESULTS = pd.concat([CV_RESULTS ,pd.DataFrame({'df_reduced_2deg_reduced_b':cv_scores(X_f_2_b,Y)},
                                                 index=['MAE','MSE', 'MAPE','R^2','F-stat','N_Preds'])], axis =  1)

CV_RESULTS = pd.concat([CV_RESULTS ,pd.DataFrame({'df_reduced_2deg_reduced_s':cv_scores(X_f_2_s,Y)},
                                                 index=['MAE','MSE', 'MAPE','R^2','F-stat','N_Preds'])], axis =  1)

CV_RESULTS

,Pol.F_2deg,Pol.F_3deg,2deg_reduced_by_f,2deg_reduced_by_b,2deg_reduced_by_s,3deg_reduced_by_f,3deg_reduced_by_b,df_reduced_2deg,df_reduced_3deg,df_reduced_2deg_reduced_f,df_reduced_2deg_reduced_b,df_reduced_2deg_reduced_s
MAE,13.359,25.423,14.015,13.199,14.159,17.626,20.192,14.113,14.096,14.167,14.070,14.159
MSE,332.600,1981.100,358.800,323.200,367.400,599.800,1273.700,366.100,368.400,367.800,363.700,367.400
MAPE,7.744,14.914,8.098,7.645,8.177,10.183,11.625,8.148,8.128,8.181,8.119,8.177
R^2,0.563,-1.663,0.529,0.575,0.518,0.214,-0.670,0.520,0.516,0.517,0.523,0.518
F-stat,54.380,-17.560,135.640,111.910,204.570,13.490,-25.190,86.860,57.220,190.100,153.500,204.570
N_Preds,62.000,92.000,22.000,32.000,14.000,53.000,42.000,33.000,49.000,15.000,19.000,14.000


In [43]:
# Reducing the Reduced Dataset with Polynomial Features of 3rd degree via forward selection
X_f_3_f = X_f_3[forward_selection(X_f_3,Y,0.01)]

In [44]:
# Reducing the Reduced Dataset with Polynomial Features of 3rd degree via backward elimination
X_f_3_b = X_f_3[backward_elimination(X_f_3,Y,0.01)]

In [45]:
# Reducing the Reduced Dataset with Polynomial Features of 3rd degree via stepwise selection
X_f_3_s = X_f_3[stepwise_selection(X_f_3,Y,0.01,0.01)]

__5.5. Running MLR models and storing results__

In [46]:
CV_RESULTS = pd.concat([CV_RESULTS ,pd.DataFrame({'df_reduced_3deg_reduced_f':cv_scores(X_f_3_f,Y)},
                                                 index=['MAE','MSE', 'MAPE','R^2','F-stat','N_Preds'])], axis =  1)

CV_RESULTS = pd.concat([CV_RESULTS ,pd.DataFrame({'df_reduced_3deg_reduced_b':cv_scores(X_f_3_b,Y)},
                                                 index=['MAE','MSE', 'MAPE','R^2','F-stat','N_Preds'])], axis =  1)

CV_RESULTS = pd.concat([CV_RESULTS ,pd.DataFrame({'df_reduced_3deg_reduced_s':cv_scores(X_f_3_s,Y)},
                                                 index=['MAE','MSE', 'MAPE','R^2','F-stat','N_Preds'])], axis =  1)

CV_RESULTS

,Pol.F_2deg,Pol.F_3deg,2deg_reduced_by_f,2deg_reduced_by_b,2deg_reduced_by_s,3deg_reduced_by_f,3deg_reduced_by_b,df_reduced_2deg,df_reduced_3deg,df_reduced_2deg_reduced_f,df_reduced_2deg_reduced_b,df_reduced_2deg_reduced_s,df_reduced_3deg_reduced_f,df_reduced_3deg_reduced_b,df_reduced_3deg_reduced_s
MAE,13.359,25.423,14.015,13.199,14.159,17.626,20.192,14.113,14.096,14.167,14.070,14.159,14.135,14.008,14.118
MSE,332.600,1981.100,358.800,323.200,367.400,599.800,1273.700,366.100,368.400,367.800,363.700,367.400,366.400,359.100,365.600
MAPE,7.744,14.914,8.098,7.645,8.177,10.183,11.625,8.148,8.128,8.181,8.119,8.177,8.163,8.072,8.153
R^2,0.563,-1.663,0.529,0.575,0.518,0.214,-0.670,0.520,0.516,0.517,0.523,0.518,0.519,0.529,0.520
F-stat,54.380,-17.560,135.640,111.910,204.570,13.490,-25.190,86.860,57.220,190.100,153.500,204.570,179.590,110.320,206.220
N_Preds,62.000,92.000,22.000,32.000,14.000,53.000,42.000,33.000,49.000,15.000,19.000,14.000,16.000,27.000,14.000


## 6. CONCLUSION

The best model (error wise) is produced when we expand the original dataset by adding Polynomial Features of 2nd degree and then reducing it via backward elimination.

The rest of the models do not show any remarkable improvement.

It is also obvious that most models that contain predictors with Polynomial Features of 3rd degree are worse that the rest.

__Saving the Results__

In [47]:
CV_RESULTS.to_csv(r'PolynomialRegressionModels_CV_Results.csv', index=True, index_label = 'Metric')